In [ ]:
import torch
import torchvision
from torchvision import transforms

import pickle
import numpy as np
from PIL import Image
from urllib.request import urlopen

torch.set_grad_enabled(False);

# Utiliser un réseau pré-entrainé

In [ ]:
# Pour charger un CNN déjà entrainé
cnn_model = torchvision.models.resnet34(pretrained=True)
cnn_model.eval()

# Pour définir un dictionnaire python qui associe à chaque index le nom de la classe correspondante 
idx_to_class = np.array(list(pickle.load(urlopen('https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl')).values()))

# Pour transformer l'image en "Tenseur" que le modèle peut utiliser
transform = transforms.Compose([
    transforms.Resize(300),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Lambda(lambda x: x.unsqueeze(0))
])

In [ ]:
# Decrit le modele
print(cnn_model)

In [ ]:
# Choisissez un image sur internet

# URL d'images : volcan, chat
url_volcan = "https://www.reunion.fr/sites/crt-reunion/files/styles/ogimage/public/visuel/volcan10eruption-creditirt-sergegelabertdts1220141.jpg?itok=wA6gronH"
url_chat = "https://farm1.static.flickr.com/196/443645811_8c4bb1af50.jpg"

url = url_volcan
image = Image.open(urlopen(url))
image

In [ ]:
# Transforme l'image en Tenseur
tensor = transform(image)

# Calcule les probabilités pour chaque classe en utilisant le CNN
outputs = cnn_model(tensor)[0]
probabilities = outputs.softmax(0)

# Affiche la classe la plus probable
max_prob, max_index = probabilities.topk(1)
max_class = idx_to_class[max_index]
print("Prediction: {} - Probability: {:.1%}".format(max_class, max_prob.item()))

In [ ]:
# Affiche les 5 classes les plus probables
pbs, ids = probabilities.topk(5)

for elmt in zip(idx_to_class[ids], pbs):
    print("{:20} : {:.1%}".format(*elmt))